In [11]:
cd /media/sf_datasets/Smarter\ Devices/BLUED_extracted/BLUED-TK

/media/sf_datasets/Smarter Devices/BLUED_extracted/BLUED-TK


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

sess = tf.InteractiveSession()

In [13]:
#reading the dataframe
df=pd.read_table('events/peaks.csv',index_col=0,sep='\t')

#converting the string to numpy array
def makeArray(text):
    text=text.replace("["," ").strip()
    return np.fromstring(text,sep=' ')

df.features=df.features.apply(makeArray)

In [14]:
#finding the single event devices
freq=df.groupby('label').count()
# freq=freq.sort_values(by='features')
single_freq=freq[freq.features==1].index
# single_freq

In [15]:
#eliminating all the single event devices
for i in single_freq:
    df=(df[df['label']!=i])
# #eliminating refrigerator
# df=(df[df['label']!=111])
# #eliminating monitor
# df=(df[df['label']!=140])
# print(len(df))
# printing the values by count
# df.groupby('label').count().sort_values(by='features')

In [16]:

# #what's causing the problem
# for i in df.label.unique():
#     print(i)
#     val=df[df.label==i].features
#     for v in val:
#         plt.plot(v)
#         plt.show()

In [17]:
inputs=70
layer1=100
layer2=100
outputs=34

total_events=len(df)

x = tf.placeholder(tf.float32, shape=[None, inputs])
y = tf.placeholder(tf.float32, shape=[None, outputs])


W = tf.Variable(tf.zeros([inputs,outputs]))
b = tf.Variable(tf.zeros([outputs]))

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([inputs, layer1])),
    'h2' :tf.Variable(tf.random_normal([layer1, layer2])),
    'ho' :tf.Variable(tf.random_normal([layer2, outputs])),    
}

biases = {
    'b1': tf.Variable(tf.random_normal([layer1])),
    'b2': tf.Variable(tf.random_normal([layer2])),
    'bo': tf.Variable(tf.random_normal([outputs])),

}



In [18]:
#output layer
def ann(x,weights,biases):
    layer1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer2=tf.add(tf.matmul(layer1, weights['h2']), biases['b2'])
    out_layer = tf.add(tf.matmul(layer2, weights['ho']), biases['bo'])
    return out_layer
    
pred = ann(x, weights, biases)

In [19]:
feature_matrix=np.empty((total_events,inputs))
label_matrix=np.zeros((total_events,outputs))

ll=np.empty((total_events))
count=0
for i in df.index:
        feature_matrix[count]=df.loc[i].features
        ll[count]=df.loc[i].label
        count+=1
        
device_list=list(df.label.unique())
for i in range(0,len(ll)):
    label_matrix[i,device_list.index(ll[i])]=1
print(feature_matrix.shape)
print(label_matrix.shape)

(2226, 70)
(2226, 34)


In [10]:
print(feature_matrix)

[[  4.65933250e-01   5.29614250e-01   5.93295250e-01 ...,  -1.27584998e+01
   -1.27160457e+01  -1.26735917e+01]
 [ -6.11480500e-02  -8.22340500e-02   1.07536950e-01 ...,   4.02736950e-01
    4.44907950e-01   1.91879950e-01]
 [  2.11209250e-01   4.44706250e-01   5.08387250e-01 ...,  -1.27584998e+01
   -1.27797268e+01  -1.27372728e+01]
 ..., 
 [ -1.16748500e-01  -7.42945000e-02  -1.37975500e-01 ...,  -5.30675000e-02
   -1.06135000e-02  -1.06135000e-02]
 [ -3.50245500e-02   1.56018450e-01   1.34791450e-01 ...,   1.77245450e-01
    2.86564500e-02   9.23374500e-02]
 [  7.48539000e-02   9.59399000e-02   1.15969000e-02 ...,   2.43539900e-01
   -3.05741000e-02   3.26829000e-02]]


In [ ]:
# Parameters
learning_rate = 0.003
training_epochs = 1000
display_step = training_epochs/10

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        _, c = sess.run([optimizer, cost], feed_dict={x: feature_matrix,
                                                      y: label_matrix})
        avg_cost = c 
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        if not epoch%display_step:
            print ("Accuracy:%f" %(accuracy.eval({x: feature_matrix, y: label_matrix})*100))